# DNBLab Jupyter Notebook zur Datenextraktion aus XML-Datei

Dieses DNBLab-Tutorial beschreibt das beispielhafte Laden einer XML-Datei im MARC21-Format. In der Jupyter Notebook Umgebung kann der dokumentierte Code direkt ausgeführt und angepasst werden. Das Tutorial umfasst das Durchsuchen der Daten in MARC21-xml und Ausgabe ausgewählter Elemente als CSV-Datei. 

## Arbeitsumgebung einrichten <a class="anchor" id="Teil1"></a>

In [1]:
import pandas as pd
import lxml.etree as ET
from bs4 import BeautifulSoup as soup
import unicodedata

## Laden der Daten aus Datei 

In [2]:
# open from file: 
with open("d028.xml", "r", encoding="utf-8") as f2:
    result = f2.readlines()

## Parsen und Anzeige des ersten Datensatzes

In [9]:
content = "".join(result)
xml = soup(content, "lxml")
myquery = xml.find_all('record', {'type':'Bibliographic'})
print(len(myquery))
print(myquery[0].prettify())

196
<record type="Bibliographic" xmlns="http://www.loc.gov/MARC21/slim">
 <leader>
  00000nam a2200000 c 4500
 </leader>
 <controlfield tag="001">
  1163616249
 </controlfield>
 <controlfield tag="003">
  DE-101
 </controlfield>
 <controlfield tag="005">
  20240212135623.0
 </controlfield>
 <controlfield tag="007">
  cr||||||||||||
 </controlfield>
 <controlfield tag="008">
  180731r20181964gw |||||o|||| 00||||ger
 </controlfield>
 <datafield ind1=" " ind2=" " tag="015">
  <subfield code="a">
   19,O01
  </subfield>
  <subfield code="2">
   dnb
  </subfield>
 </datafield>
 <datafield ind1="7" ind2=" " tag="016">
  <subfield code="2">
   DE-101
  </subfield>
  <subfield code="a">
   1163616249
  </subfield>
 </datafield>
 <datafield ind1="7" ind2=" " tag="024">
  <subfield code="2">
   urn
  </subfield>
  <subfield code="a">
   urn:nbn:de:101:1-2018090918265168878989
  </subfield>
 </datafield>
 <datafield ind1=" " ind2=" " tag="035">
  <subfield code="a">
   (DE-599)DNB1163616249
  </s

## Suche nach ausgewählten Informationen in den einzelnen Datensätzen

In [10]:
#Funktion für Titeldaten in MARC21

def parse_record_marc(item):

    ns = {"marc":"http://www.loc.gov/MARC21/slim"}
    xml = ET.fromstring(unicodedata.normalize("NFC", str(item)))
    
    #idn
    idn = xml.findall("marc:controlfield[@tag = '001']", namespaces=ns)
    try:
        idn = idn[0].text
    except:
        idn = 'N/A' 
        
    
    #Titel 
    title = xml.findall("marc:datafield[@tag = '245']/marc:subfield[@code = 'a']", namespaces=ns)
    title2 = xml.findall("marc:datafield[@tag = '245']/marc:subfield[@code = 'b']", namespaces=ns)
    
    if title and not title2:
        titletext = title[0].text
    elif title and title2:     
        titletext = title[0].text + " " + title2[0].text   #Hier wird extra nur mit einem Leerzeichen getrennt statt eines Separators
    else:
        titletext = "N/A"
    
    
    #date
    date = xml.findall("marc:datafield[@tag = '264']/marc:subfield[@code = 'c']", namespaces=ns)
    date2 = xml.findall("marc:controlfield[@tag = '008']", namespaces=ns)
    if date:
        date = date[0].text
    elif date2:
        date = date2[0].text
        date = date[7:11]
    else:    
        date = 'N/A'
          
    #lang:
    lang = xml.findall("marc:datafield[@tag = '041']/marc:subfield[@code = 'a']", namespaces=ns)
    try:
        lang = lang[0].text
    except:
        lang = 'N/A'
    
    
    #URN: 
    urn = "N/A"
    for urn in xml.findall("marc:datafield[@tag = '024']/marc:subfield[@code = 'a']", namespaces=ns):
        if urn.text.startswith("urn:nbn"):
            urn = urn.text
        else: 
            urn = "N/A"
    
    
    meta_dict = {"idn":idn, "title": titletext, "date":date, "lang":lang, "urn":urn} 
               
    return meta_dict
    

## Sammeln der extrahierten Informationen aus allen Datensätzen

In [12]:
output = [parse_record_marc(record) for record in myquery]

## Anzeige der Daten in einer Tabelle/ Dataframe

In [13]:
df = pd.DataFrame(output)
df

,idn,title,date,lang,urn
0,1163616249,13 Konkrete Vordemberge-Gildewart ... ; [Kunst...,2018,ger,urn:nbn:de:101:1-2018090918265168878989
1,1163613266,200 Jahre Frommann-Verlag [1727-1927],2018,ger,urn:nbn:de:101:1-2018090917393540952215
2,1163614645,60 Fotos,2018,ger,urn:nbn:de:101:1-2018101418234208736122
3,1163615773,Abendgesang (Neila) Letzte Gedichte,2018,ger,urn:nbn:de:101:1-2018090918272331231107
4,1163615218,"Abstraktion in der Malerei Kandinsky, Feininge...",2018,ger,urn:nbn:de:101:1-2018090918270991257647
...,...,...,...,...,...
191,1163612235,Willy Baumeister [Ausstellung Willy Baumeister...,2018,ger,urn:nbn:de:101:1-2018102211013265416631
192,1163615420,Zeitberichte und Dokumente,2018,ger,urn:nbn:de:101:1-2018101418250397989578
193,1163614211,Ziele des Schriftunterrichts Ein Beitr. zur mo...,2018,ger,urn:nbn:de:101:1-2018090917544063233045
194,1163614912,Zum neuen Bauen Vorträge,2018,ger,urn:nbn:de:101:1-2018092317065678945974


## Export der Daten als CSV-Datei

In [14]:
df.to_csv("Dump_Titel.csv", index=False)